In [1]:
import openai
import llm2geneset
import pandas as pd
import random

aclient = openai.AsyncClient()

In [2]:
al002 = pd.read_csv("data/Stats.TECAN.AL002_vs_isotype.txt", sep="\t")
al002 = al002[al002["PercentDetectedSamples"] > 0.9]

In [3]:
al002_sig = al002[(al002["padj"] < 0.1) ]
genes = al002_sig["gene"].to_list()

In [4]:
random.seed(138337)
random.shuffle(genes)

In [5]:
print("\n".join(genes))

LILRB2
MX1
GSN
MFGE8
CCL3
GPD1
TRPV2
DHCR24
TNFAIP3
MATK
CXCL8
CHIT1
TPRKB
RASSF4
SMC6
FOS
RHOBTB2
RNF19B
FBP1
SLC11A1
DCSTAMP
HSD3B7
OAS2
TGM2
MCEMP1
FAM20C
ACP5
CD9
AP5B1


In [6]:
context_str = ""
llm2geneset_no_context_res = await llm2geneset.gs_proposal_bench(aclient, [genes], model="gpt-4o", context=context_str,
                                                                 n_pathways=100, n_retry=3)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:37<00:00, 37.08s/it]


In [7]:
df = llm2geneset_no_context_res[0]["ora_results"]
df.to_csv("outputs/no_context_AL002.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.05]

,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter,in_toks,out_toks
1,Negative regulation of viral genome replication,0.068966,0.001109,0.090909,62.213166,0.000469,0.030940,"OAS2,MX1","IFIT2,PKR,ZAP,ISG15,EIF2AK2,APOBEC3G,OAS2,IFIT...",29,22,2,160,205
67,Regulation of inflammatory response,0.068966,0.001512,0.066667,45.622989,0.000877,0.030940,"FOS,CXCL8","TLR4,CASP1,CTNNB1,CXCL8,TNF,STAT3,MAPK14,NLRP3...",29,30,2,158,394
26,Osteoclast differentiation,0.068966,0.001612,0.062500,42.771552,0.000998,0.030940,"DCSTAMP,FOS","NFATC2,PTK2B,ITGAV,CTSK,ITGB3,TNFRSF11B,RANKL,...",29,32,2,158,424
22,Antiviral defense,0.068966,0.002066,0.048780,33.382675,0.001637,0.038051,"OAS2,MX1","RIGI,TLR3,CASP12,CASP1,APOBEC3,CASP5,PKR,CASP4...",29,41,2,156,529


In [8]:
context_str = "microglia treated with a TREM2 agonist antibody"
llm2geneset_context_res = await llm2geneset.gs_proposal_bench(aclient, [genes], model="gpt-4o", context=context_str,
                                                              n_pathways=100, n_retry=3)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:25<00:00, 25.49s/it]


In [10]:
df = llm2geneset_context_res[0]["ora_results"]
df.to_csv("outputs/with_context_AL002.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.05]

,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter,in_toks,out_toks
17,Negative regulation of viral genome replication,0.068966,0.001159,0.086957,59.508246,0.000513,0.022711,"OAS2,MX1","IFIT2,PKR,TREX1,OASL,ISG15,SAMHD1,OAS2,MX2,IFI...",29,23,2,160,208
53,Leukocyte chemotaxis,0.068966,0.001461,0.068966,47.196195,0.000819,0.022711,"CXCL8,CCL3","Selectin P,CCR5,PECAM1,CXCL2,CXCL1,CXCL12,CXCL...",29,29,2,160,259
60,Virus defense,0.068966,0.001512,0.066667,45.622989,0.000877,0.022711,"OAS2,MX1","RIGI,TLR3,IFI44,MAVS,IFIT2,PKR,TLR7,TRIM21,ISG...",29,30,2,152,391
3,Antiviral mechanism by IFN-stimulated genes,0.068966,0.001562,0.064516,44.151279,0.000937,0.022711,"OAS2,MX1","DENV2,IFIT2,PKR,TRIM79,IFI27,CGAS,RIG-I,TRIM21...",29,31,2,170,404
4,Cytokine-mediated signaling pathway,0.068966,0.002368,0.042553,29.121056,0.002146,0.041629,"CXCL8,CCL3","IFNGR1,TLR4,IL18,STAT4,CXCL8,IL2,STAT6,CCL3,TN...",29,47,2,162,614
6,Type I interferon signaling pathway,0.068966,0.002721,0.037037,25.346105,0.002823,0.045633,"OAS2,MX1","IFNA13,GBP2,IFNA4,IFI44,HLA-A,IFNA14,IFNA2,IFN...",29,54,2,160,480


In [11]:
gmt = llm2geneset.read_gmt("libs_human/gmt/KEGG_2021_Human.txt")
set_descr = gmt["descr"]
gene_sets = gmt["genes"]
df = llm2geneset.simple_ora(genes, set_descr, gene_sets)
df.to_csv("outputs/KEGG_AL002.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.05]

,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter
251,Rheumatoid arthritis,0.137931,0.004686,0.043011,29.434186,0.000010,0.003157,"FOS,CXCL8,ACP5,CCL3","ITGAL,TCIRG1,ATP6V0D1,HLA-DOA,TGFB2,CXCL2,HLA-...",29,93,4
161,Measles,0.137931,0.007004,0.028777,19.693376,0.000048,0.007686,"OAS2,TNFAIP3,FOS,MX1","CCNE1,CDKN1B,TRAF3,BCL2,CDK4,CASP8,HSPA6,JAK3,...",29,139,4
173,NOD-like receptor signaling pathway,0.137931,0.009120,0.022099,15.123643,0.000134,0.014265,"OAS2,TNFAIP3,TRPV2,CXCL8","GBP2,TRAF3,BCL2,CXCL2,CASP8,NOD1,TAB3,TAB1,TXN...",29,181,4
136,IL-17 signaling pathway,0.103448,0.004736,0.031915,21.840792,0.000345,0.021238,"FOS,TNFAIP3,CXCL8","IL17F,TRAF3,LCN2,CXCL2,CASP8,FOSL1,FADD,TAB3,S...",29,94,3
64,Coronavirus disease,0.137931,0.011690,0.017241,11.799049,0.000345,0.021238,"OAS2,FOS,CXCL8,MX1","TRAF3,RPL3L,RPS26,MASP1,RPL36AL,CGAS,TLR7,RPL4...",29,232,4
50,Chagas disease,0.103448,0.005140,0.029412,20.127789,0.000439,0.021238,"FOS,CXCL8,CCL3","TGFB2,SERPINE1,CASP8,FADD,CCL5,AKT3,TLR6,IRAK4...",29,102,3
290,Toll-like receptor signaling pathway,0.103448,0.005240,0.028846,19.740716,0.000465,0.021238,"FOS,CXCL8,CCL3","TRAF3,CASP8,TAB1,FADD,TLR7,CCL5,TLR8,AKT3,MAP2...",29,104,3
191,Osteoclast differentiation,0.103448,0.006399,0.023622,16.165626,0.000832,0.033260,"LILRB2,FOS,ACP5","TGFB2,LILRA1,PPP3R1,SIRPA,FOSL1,TAB1,PPARG,TRE...",29,127,3
